## TODO
1. add features
2. change targets
3. model
4. loss_fn
5. tune the parameters

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

## packages and settings

In [ ]:
import numpy as np 
import random
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
import pandas as pd
import os
import copy
import sys

import torch
import torch.nn as nn
from torch.autograd import Variable
from torch.utils.data import DataLoader
from torch.nn import CrossEntropyLoss, MSELoss
from torch.nn.modules.loss import _WeightedLoss
import torch.nn.functional as F
from sklearn.metrics import log_loss, roc_auc_score

from sklearn.model_selection import GroupKFold
from tqdm import tqdm

pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 100)

seed = 123
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.backends.cudnn.deterministic = True

if not os.path.exists("results"):
    os.mkdir("results")

TRAINING = True
read_path = '/kaggle/input/jane-street-market-prediction/train.csv'
model_path = "/kaggle/input/skeleton-with-pytorch/results/123/best_model"
save_path = os.path.join("results", str(seed))

device = torch.device("cuda:0")
if not os.path.exists(save_path):
    os.mkdir(save_path)
    
# train = pd.read_csv(read_path)

In [ ]:
TRAINING = True

## preprocess the data

In [ ]:
train = pd.read_csv(read_path)
train = train.query('date > 85').reset_index(drop = True) 
features = [c for c in train.columns if 'feature' in c]

# preprocess the features
f_mean = train[features].mean()
train = train.loc[train.weight > 0].reset_index(drop = True)
train[features] = train[features].fillna(f_mean)

train = train.astype("float32")
train['action'] = (train['resp'] > 0).astype('int')
train['action1'] = (train['resp_1'] > 0).astype('int')
train['action2'] = (train['resp_2'] > 0).astype('int')
train['action3'] = (train['resp_3'] > 0).astype('int')
train['action4'] = (train['resp_4'] > 0).astype('int')

# targets = ['resp']
# targets = ['action']
targets = ['action', 'action_1', 'action_2', 'action_3', 'action_4']
# train[targets] = (train[targets]>0).astype('int')


def add_features(df, features):
    new_features = copy.deepcopy(features)
    
    # todo
    df["cross_1_2"] = df["feature_1"] / (df["feature_2"] + 1e-5)
    df["cross_41_42_43"] = df["feature_41"] + df["feature_42"] + df["feature_43"]
    new_features.extend(["cross_1_2", "cross_41_42_43"])

    return df, new_features

train, train_features = add_features(train, features)


# to do: update the mean online
# f_mean = f_mean.values
# np.save(os.path.join(save_path, 'f_mean.npy'), f_mean)

In [ ]:
train = pd.read_csv(read_path)
valid = train.loc[(train.date >= 450) & (train.date < 500)].reset_index(drop=True)
valid.fillna(train.mean(), inplace=True)
del train
valid['action'] = (valid['resp'] > 0).astype('int')
valid['action_1'] = (valid['resp_1'] > 0).astype('int')
valid['action_2'] = (valid['resp_2'] > 0).astype('int')
valid['action_3'] = (valid['resp_3'] > 0).astype('int')
valid['action_4'] = (valid['resp_4'] > 0).astype('int')
valid['cross_1_2'] = valid['feature_1'] / (valid['feature_2'] + 1e-5)
valid['cross_41_42_43'] = valid['feature_41'] + valid['feature_42'] + valid['feature_43']



In [ ]:
BATCH_SIZE = 8192
class MarketDataset:
    def __init__(self, df):
        self.features = df[train_features].values

        self.label = df[targets].values.reshape(-1, len(targets))

    def __len__(self):
        return len(self.label)

    def __getitem__(self, idx):
        return {
            'features': torch.tensor(self.features[idx], dtype=torch.float),
            'label': torch.tensor(self.label[idx], dtype=torch.float)
        }

valid_set = MarketDataset(valid)
valid_loader = DataLoader(valid_set, batch_size=BATCH_SIZE, shuffle=False, num_workers=4)

def inference_fn1(model, dataloader, device):
    model.eval()
    preds = []

    for data in dataloader:
        features = data['features'].to(device)

        with torch.no_grad():
            outputs = model(features)

        preds.append(outputs.sigmoid().detach().cpu().numpy())

    preds = np.concatenate(preds).reshape(-1, len(targets))

    return preds

## the dataset and model(resnet)

In [ ]:
class MyDataset:
    def __init__(self, df, features, targets):
        self.features = df[features].values
        
        # preprocess the labels
        # self.labels = (df[targets] > 0).astype('int').values
        self.labels = df[targets].values
        self.weights = df['weight'].values

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        feat_ = torch.tensor(self.features[idx], dtype=torch.float)
        label_ = torch.tensor(self.labels[idx], dtype=torch.float)
        weight_ = torch.tensor(self.weights[idx], dtype=torch.float)
        
        return feat_, label_, weight_

In [ ]:
class Model1(nn.Module):
    def __init__(self, features, targets):
        super(Model, self).__init__()
        self.batch_norm0 = nn.BatchNorm1d(len(features))
        self.dropout0 = nn.Dropout(0.10143786981358652)

        hidden_size = 256
        self.dense1 = nn.Linear(len(features), 384)
        self.batch_norm1 = nn.BatchNorm1d(384)
        self.dropout1 = nn.Dropout(0.19720339053599725)

        self.dense2 = nn.Linear(384, 896)
        self.batch_norm2 = nn.BatchNorm1d(896)
        self.dropout2 = nn.Dropout(0.2703017847244654)

        self.dense3 = nn.Linear(896, 896)
        self.batch_norm3 = nn.BatchNorm1d(896)
        self.dropout3 = nn.Dropout(0.23148340929571917)

        self.dense4 = nn.Linear(896, 394)
        self.batch_norm4 = nn.BatchNorm1d(394)
        self.dropout4 = nn.Dropout(0.2357768967777311)

        self.dense5 = nn.Linear(394, len(targets))

        self.Relu = nn.ReLU(inplace=True)
        self.PReLU = nn.PReLU()
        self.LeakyReLU = nn.LeakyReLU(negative_slope=0.01, inplace=True)
        # self.GeLU = nn.GELU()
        self.RReLU = nn.RReLU()

    def forward(self, x):
        x = self.batch_norm0(x)
        x = self.dropout0(x)

        x = self.dense1(x)
        x = self.batch_norm1(x)
        x = x * torch.sigmoid(x)
        x = self.dropout1(x)

        x = self.dense2(x)
        x = self.batch_norm2(x)
        x = x * torch.sigmoid(x)
        x = self.dropout2(x)
        
        x = self.dense3(x)
        x = self.batch_norm3(x)
        x = x * torch.sigmoid(x)
        x = self.dropout3(x)
        
        x = self.dense4(x)
        x = self.batch_norm4(x)
        x = x * torch.sigmoid(x)
        x = self.dropout4(x)

        x = self.dense5(x)

        return x
    
    
    
class SmoothBCEwLogits(_WeightedLoss):
    def __init__(self, weight=None, reduction='mean', smoothing=0.0):
        super().__init__(weight=weight, reduction=reduction)
        self.smoothing = smoothing
        self.weight = weight
        self.reduction = reduction

    @staticmethod
    def _smooth(targets:torch.Tensor, n_labels:int, smoothing=0.0):
        assert 0 <= smoothing < 1
        with torch.no_grad():
            targets = targets * (1.0 - smoothing) + 0.5 * smoothing
        return targets

    def forward(self, inputs, targets, weights=None):
        targets = SmoothBCEwLogits._smooth(targets, inputs.size(-1),
            self.smoothing)
        loss = F.binary_cross_entropy_with_logits(inputs, targets, weights)

        if  self.reduction == 'sum':
            loss = loss.sum()
        elif  self.reduction == 'mean':
            loss = loss.mean()

        return loss

    
class EarlyStopping:
    def __init__(self, patience=7, mode="max", delta=0.):
        self.patience = patience
        self.counter = 0
        self.mode = mode
        self.best_score = None
        self.early_stop = False
        self.delta = delta
        if self.mode == "min":
            self.val_score = np.Inf
        else:
            self.val_score = -np.Inf

    def __call__(self, epoch_score, model, model_path):

        if self.mode == "min":
            score = -1.0 * epoch_score
        else:
            score = np.copy(epoch_score)

        if self.best_score is None:
            self.best_score = score
            self.save_checkpoint(epoch_score, model, model_path)
        elif score < self.best_score: #  + self.delta
            self.counter += 1
            print('EarlyStopping counter: {} out of {}'.format(self.counter, self.patience))
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_score = score
            # ema.apply_shadow()
            self.save_checkpoint(epoch_score, model, model_path)
            # ema.restore()
            self.counter = 0

    def save_checkpoint(self, epoch_score, model, model_path):
        if epoch_score not in [-np.inf, np.inf, -np.nan, np.nan]:
            print('Validation score improved. Saving model!')
            torch.save(model.state_dict(), model_path)
        self.val_score = epoch_score

In [ ]:
class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        self.batch_norm0 = nn.BatchNorm1d(len(train_features))
        self.dropout0 = nn.Dropout(0.2)

        dropout_rate = 0.2
        hidden_size = 256
        self.dense1 = nn.Linear(len(train_features), hidden_size)
        self.batch_norm1 = nn.BatchNorm1d(hidden_size)
        self.dropout1 = nn.Dropout(dropout_rate)

        self.dense2 = nn.Linear(hidden_size+len(train_features), hidden_size)
        self.batch_norm2 = nn.BatchNorm1d(hidden_size)
        self.dropout2 = nn.Dropout(dropout_rate)

        self.dense3 = nn.Linear(hidden_size+hidden_size, hidden_size)
        self.batch_norm3 = nn.BatchNorm1d(hidden_size)
        self.dropout3 = nn.Dropout(dropout_rate)

        self.dense4 = nn.Linear(hidden_size+hidden_size, hidden_size)
        self.batch_norm4 = nn.BatchNorm1d(hidden_size)
        self.dropout4 = nn.Dropout(dropout_rate)

        self.dense5 = nn.Linear(hidden_size+hidden_size, len(targets))

        self.Relu = nn.ReLU(inplace=True)
        self.PReLU = nn.PReLU()
        self.LeakyReLU = nn.LeakyReLU(negative_slope=0.01, inplace=True)
        # self.GeLU = nn.GELU()
        self.RReLU = nn.RReLU()

    def forward(self, x):
        x = self.batch_norm0(x)
        x = self.dropout0(x)

        x1 = self.dense1(x)
        x1 = self.batch_norm1(x1)
        # x = F.relu(x)
        # x = self.PReLU(x)
        x1 = self.LeakyReLU(x1)
        x1 = self.dropout1(x1)

        x = torch.cat([x, x1], 1)

        x2 = self.dense2(x)
        x2 = self.batch_norm2(x2)
        # x = F.relu(x)
        # x = self.PReLU(x)
        x2 = self.LeakyReLU(x2)
        x2 = self.dropout2(x2)

        x = torch.cat([x1, x2], 1)

        x3 = self.dense3(x)
        x3 = self.batch_norm3(x3)
        # x = F.relu(x)
        # x = self.PReLU(x)
        x3 = self.LeakyReLU(x3)
        x3 = self.dropout3(x3)

        x = torch.cat([x2, x3], 1)

        x4 = self.dense4(x)
        x4 = self.batch_norm4(x4)
        # x = F.relu(x)
        # x = self.PReLU(x)
        x4 = self.LeakyReLU(x4)
        x4 = self.dropout4(x4)

        x = torch.cat([x3, x4], 1)

        x = self.dense5(x)

        return x

## Train

### utility

In [ ]:
def train_fn(model, optimizer, scheduler, loss_fn, dataloader, device):
    model.train()
    final_loss = 0

    for feature, label, weight in dataloader:
        feature = feature.to(device)
        label = label.to(device)
        weight = weight.to(device)
        optimizer.zero_grad()
        outputs = model(feature)
        
        
        loss = loss_fn(outputs.reshape(-1, 1), label.reshape(-1, 1)) #  weight.reshape(-1,1)
        
        loss.backward()
        optimizer.step()
        if scheduler:
            scheduler.step()

        final_loss += loss.item()

    final_loss /= len(dataloader)
    return final_loss
        
    
def inference_fn(model, dataloader, device):
    model.eval()
    preds = []
    labels = []

    for feature, label, weight in dataloader:
        feature = feature.to(device)
        # label = label.to(device)
        with torch.no_grad():
            outputs = model(feature)
            preds.append(outputs.cpu().numpy())
            labels.append(label.cpu().numpy())
    
    preds = np.concatenate(preds, axis=0)
    labels = np.concatenate(labels, axis=0)

    return preds, labels
    
    
def utility_score(date, weight, resp, action):    
    values = weight * resp * action
    to_bincount = {}

    for d, v in zip(date, values):
        to_bincount.setdefault(d, []).append(v)

    Pi = []
    for val in to_bincount.values():
        Pi.append(np.sum(val))
    Pi = np.array(Pi)
    count_i = len(np.unique(date))
    t = np.sum(Pi) / np.sqrt(np.sum(Pi ** 2)) * np.sqrt(250 / count_i)
    u = np.clip(t, 0, 6) * np.sum(Pi)
    
    return u

    
def loss_mse(preds, targets):
    
    return ((preds-targets)**2).mean()


def loss_ce(preds, targets, weight=None):
    
    return F.binary_cross_entropy_with_logits(preds, targets, weight)

In [ ]:
# test_preds, true_test_labels = inference_fn(model, test_loader, device)
# test_preds = np.where(test_preds >= 0.5, 1, 0).astype(int)
# u_score = utility_score(date=test.date.values.reshape(-1),
#                         weight=test.weight.values.reshape(-1),
#                         resp=test.resp.values.reshape(-1),
#                         action=action1)

## valid

In [ ]:
NFOLDS = 5
models = []
#tmp = np.zeros(len(feat_cols))
for _fold in range(NFOLDS):
    torch.cuda.empty_cache()
    model = Model()
    model.to(device)
    model_weights = f"/kaggle/input/mlp012003weights/online_model{_fold}.pth"
    model.load_state_dict(torch.load(model_weights))
    model.eval()
    models.append(model)

In [ ]:
valid_pred = np.zeros((len(valid), len(targets)))
for model in models:
    valid_pred += inference_fn1(model, valid_loader, device) / len(models)

valid_pred = np.median(valid_pred, axis=1)

In [ ]:
best_threshold, best_u_score = 0.5, 0

thres = float(5000) / 10000
slice_valid_pred = valid_pred.copy()
slice_valid_pred = np.where(slice_valid_pred >= thres, 1, 0).astype(int)
valid_u_score = utility_score(date=valid.date.values, weight=valid.weight.values,
                                       resp=valid.resp.values, action=slice_valid_pred)
print(f'thresold={thres:.4f}, valid_u_score={valid_u_score:.4f}')


In [ ]:
f_mean = np.load(f'/kaggle/input/mlp012003weights/f_mean_online.npy')

if True:
    import janestreet
    env = janestreet.make_env()
    env_iter = env.iter_test()

    for (test_df, pred_df) in tqdm(env_iter):
        if test_df['weight'].item() > 0:
            x_tt = test_df.loc[:, features].values
            if np.isnan(x_tt.sum()):
                x_tt = np.nan_to_num(x_tt) + np.isnan(x_tt) * f_mean
                
            cross_1_2 = x_tt[:, 1] / (x_tt[:, 2] + 1e-5)
            cross_41_42_43 = x_tt[:, 41] + x_tt[:, 42] + x_tt[:, 43]
            feature_inp = np.concatenate((
                x_tt,
                np.array(cross_41_42_43).reshape(x_tt.shape[0], 1),
                np.array(cross_1_2).reshape(x_tt.shape[0], 1),
            ), axis=1)

            # torch_pred
            torch_pred = np.zeros((1, len(targets)))
            for model in models:
                torch_pred += model(torch.tensor(feature_inp, dtype=torch.float).to(device)).sigmoid().detach().cpu().numpy() / NFOLDS
            pred = np.median(torch_pred)
            
            
            pred_df.action = np.where(pred >= 0.5, 1, 0).astype(int)
        else:
            pred_df.action = 0
        env.predict(pred_df)